# Machine Learning Models Showcase

### Load data from database

In [1]:
# imports
import pandas as pd
import sklearn as skl
import psycopg2
import config as c

# import for multiple output per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [3]:
data_sql = """
SELECT *
FROM comprehensive_dataset;
"""

#  load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)
data_df.head()

,GUID,Name,HallOfFameStatus,HallofFameClass,YearDrafted,TO_YEAR,Years_Played,HOF_Elgibility_Year,Pick,Team,...,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF
0,MIKENILES1980,Mike Niles,Not Inducted,NaN,1980,1980,1,1985,NaN,None,...,0.8,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4
1,WAYNEROBINSON1980,Wayne Robinson,Not Inducted,NaN,1980,1980,1,1985,31.0,LAL,...,3.0,72.9,1.4,2.2,3.6,1.4,0.6,0.3,1.8,7.8
2,BILLYREID1980,Billy Reid,Not Inducted,NaN,1980,1980,1,1985,182.0,GSW,...,0.7,56.4,0.5,0.6,1.0,1.2,0.6,0.1,1.3,2.8
3,ALEXBRADLEY1981,Alex Bradley,Not Inducted,NaN,1981,1981,1,1986,86.0,NYK,...,1.2,60.4,0.8,0.9,1.7,0.3,0.3,0.1,0.7,3.4
4,GARRYWITTS1981,Garry Witts,Not Inducted,NaN,1981,1981,1,1986,103.0,WSB,...,0.9,82.5,0.6,0.7,1.3,0.8,0.4,0.1,0.8,3.8


In [4]:
data_df.dtypes

GUID                    object
Name                    object
HallOfFameStatus        object
HallofFameClass        float64
YearDrafted              int64
TO_YEAR                  int64
Years_Played             int64
HOF_Elgibility_Year      int64
Pick                   float64
Team                    object
College                 object
GP                       int64
MIN                    float64
PTS                    float64
FGM                    float64
FGA                    float64
FG%                    float64
3P_Made                float64
3PA                    float64
3P%                    float64
FTM                    float64
FTA                    float64
FT%                    float64
OREB                   float64
DREB                   float64
REB                    float64
AST                    float64
STL                    float64
BLK                    float64
TOV                    float64
EFF                    float64
dtype: object

## Preprocessing 

In [5]:
# get_dummies to encode HallofFameClass column
data_df = pd.get_dummies(data_df, prefix='HOF', columns=['HallOfFameStatus'])
data_df.drop(['HOF_Not Inducted'], axis=1, inplace=True) # remove extra get_dummies column
data_df.head()

,GUID,Name,HallofFameClass,YearDrafted,TO_YEAR,Years_Played,HOF_Elgibility_Year,Pick,Team,College,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,HOF_Hall of Fame Member
0,MIKENILES1980,Mike Niles,NaN,1980,1980,1,1985,NaN,None,None,...,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4,0
1,WAYNEROBINSON1980,Wayne Robinson,NaN,1980,1980,1,1985,31.0,LAL,Virginia Polytechnic Institute and State Unive...,...,72.9,1.4,2.2,3.6,1.4,0.6,0.3,1.8,7.8,0
2,BILLYREID1980,Billy Reid,NaN,1980,1980,1,1985,182.0,GSW,University of San Francisco,...,56.4,0.5,0.6,1.0,1.2,0.6,0.1,1.3,2.8,0
3,ALEXBRADLEY1981,Alex Bradley,NaN,1981,1981,1,1986,86.0,NYK,Villanova University,...,60.4,0.8,0.9,1.7,0.3,0.3,0.1,0.7,3.4,0
4,GARRYWITTS1981,Garry Witts,NaN,1981,1981,1,1986,103.0,WSB,College of the Holy Cross,...,82.5,0.6,0.7,1.3,0.8,0.4,0.1,0.8,3.8,0


In [6]:
# confirm HOF members encoded for 0
print(data_df.loc[data_df['GUID'] == 'MICHAELJORDAN1984'])
# This confirms HOF members = 1, non members = 0

                  GUID            Name  HallofFameClass  YearDrafted  TO_YEAR  \
597  MICHAELJORDAN1984  Michael Jordan           2009.0         1984     2002   

     Years_Played  HOF_Elgibility_Year  Pick Team  \
597            19                 2007   3.0  CHI   

                          College  ...   FT%  OREB  DREB  REB  AST  STL  BLK  \
597  University of North Carolina  ...  84.5   2.0   4.5  6.5  5.9  2.4  0.8   

     TOV   EFF  HOF_Hall of Fame Member  
597  3.5  29.2                        1  

[1 rows x 31 columns]


In [7]:
data_df.dtypes

GUID                        object
Name                        object
HallofFameClass            float64
YearDrafted                  int64
TO_YEAR                      int64
Years_Played                 int64
HOF_Elgibility_Year          int64
Pick                       float64
Team                        object
College                     object
GP                           int64
MIN                        float64
PTS                        float64
FGM                        float64
FGA                        float64
FG%                        float64
3P_Made                    float64
3PA                        float64
3P%                        float64
FTM                        float64
FTA                        float64
FT%                        float64
OREB                       float64
DREB                       float64
REB                        float64
AST                        float64
STL                        float64
BLK                        float64
TOV                 

In [8]:
# drop columns that will likely error/confuse the ML models
data_df.drop(['GUID', 'Name', 'Team', 'College', # dropping object columns 
              'HallofFameClass', 'YearDrafted', 'TO_YEAR', 'HOF_Elgibility_Year', 'MIN' # non player stats
                ], axis=1, inplace=True)
data_df.head()

,Years_Played,Pick,GP,PTS,FGM,FGA,FG%,3P_Made,3PA,3P%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,HOF_Hall of Fame Member
0,1,NaN,44,2.6,1.1,3.1,34.8,0.0,0.1,50.0,...,45.9,0.6,0.7,1.3,0.3,0.2,0.0,0.6,1.4,0
1,1,31.0,81,7.9,2.9,6.3,46.0,0.0,0.1,0.0,...,72.9,1.4,2.2,3.6,1.4,0.6,0.3,1.8,7.8,0
2,1,182.0,59,3.2,1.4,3.1,45.4,0.0,0.1,0.0,...,56.4,0.5,0.6,1.0,1.2,0.6,0.1,1.3,2.8,0
3,1,86.0,39,3.5,1.4,2.6,52.4,0.0,0.0,0.0,...,60.4,0.8,0.9,1.7,0.3,0.3,0.1,0.7,3.4,0
4,1,103.0,46,2.9,1.1,1.8,58.3,0.0,0.0,50.0,...,82.5,0.6,0.7,1.3,0.8,0.4,0.1,0.8,3.8,0


In [9]:
# check for NaNs
data_df.isnull().values.any()

True

In [10]:
# where are the NaNs?
data_df.count()

Years_Played               1217
Pick                        915
GP                         1217
PTS                        1217
FGM                        1217
FGA                        1217
FG%                        1217
3P_Made                    1217
3PA                        1217
3P%                        1217
FTM                        1217
FTA                        1217
FT%                        1217
OREB                       1217
DREB                       1217
REB                        1217
AST                        1217
STL                        1217
BLK                        1217
TOV                        1217
EFF                        1217
HOF_Hall of Fame Member    1217
dtype: int64

In [11]:
# fill NaNs with 0
data_df['Pick'] = data_df['Pick'].fillna(0)
data_df.count()

Years_Played               1217
Pick                       1217
GP                         1217
PTS                        1217
FGM                        1217
FGA                        1217
FG%                        1217
3P_Made                    1217
3PA                        1217
3P%                        1217
FTM                        1217
FTA                        1217
FT%                        1217
OREB                       1217
DREB                       1217
REB                        1217
AST                        1217
STL                        1217
BLK                        1217
TOV                        1217
EFF                        1217
HOF_Hall of Fame Member    1217
dtype: int64

In [12]:
print(data_df.shape)

(1217, 22)


# Machine Learning Models

### LogReg w/ Oversampling

In [13]:
# ML imports
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
import tensorflow as tf

In [14]:
# create our features
X = data_df.drop(['HOF_Hall of Fame Member'], axis=1)

# create our target
y = data_df['HOF_Hall of Fame Member']

In [15]:
# normal train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({0: 879, 1: 33})

### RandomOverSampler

In [16]:
# import RandomOverSampler
from imblearn.over_sampling import RandomOverSampler

In [17]:
# implement random oversampling
X_resampled, y_resampled = RandomOverSampler(random_state=1).fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 879, 1: 879})

In [18]:
# logistic regression using random oversampled data
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='liblinear')

In [19]:
# display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[254,  42],
       [  0,   9]], dtype=int64)

In [20]:
# calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.9290540540540541

In [21]:
# print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.86      1.00      0.92      0.93      0.85       296
          1       0.18      1.00      0.86      0.30      0.93      0.87         9

avg / total       0.98      0.86      1.00      0.91      0.93      0.85       305



### SMOTE

In [22]:
# import SMOTE
from imblearn.over_sampling import SMOTE

In [23]:
# implement SMOTE resampling
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 879, 1: 879})

In [24]:
# logistic regression using SMOTE data
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='liblinear')

In [25]:
# calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9341216216216216

In [26]:
# display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[257,  39],
       [  0,   9]], dtype=int64)

In [27]:
# print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.87      1.00      0.93      0.93      0.86       296
          1       0.19      1.00      0.87      0.32      0.93      0.88         9

avg / total       0.98      0.87      1.00      0.91      0.93      0.86       305



### SVM SMOTE

In [28]:
# import SVMSMOTE
from imblearn.over_sampling import SVMSMOTE

In [29]:
## implement SVM SMOTE resampling
X_resampled, y_resampled = SVMSMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 879, 1: 596})

In [30]:
# logistic regression using SVM SMOTE data
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1, solver='liblinear')

In [31]:
# calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9594594594594594

In [32]:
# display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[272,  24],
       [  0,   9]], dtype=int64)

In [33]:
# print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.92      1.00      0.96      0.96      0.91       296
          1       0.27      1.00      0.92      0.43      0.96      0.93         9

avg / total       0.98      0.92      1.00      0.94      0.96      0.91       305



## Decision Tree and Neural Net

### Scale the data

In [34]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Decision Tree Model

In [35]:
# DT imports
from path import Path
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [36]:
# create the decision tree classifier instance.
model = tree.DecisionTreeClassifier()

# fit the model.
model = model.fit(X_train_scaled, y_train)

In [37]:
# DT predictions
predictions = model.predict(X_test_scaled)

In [38]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Not HOF Member", "HOF Member"], 
    columns=["Predicted Not HOF Member", "Predicted HOF Member"])
cm_df

,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,289,7
HOF Member,6,3


In [39]:
# calcuate the accuracy score.
acc_score = accuracy_score(y_test, predictions)

In [40]:
# results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted Not HOF Member,Predicted HOF Member
Not HOF Member,289,7
HOF Member,6,3


Accuracy Score : 0.9573770491803278
Classification Report
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       296
           1       0.30      0.33      0.32         9

    accuracy                           0.96       305
   macro avg       0.64      0.65      0.65       305
weighted avg       0.96      0.96      0.96       305



## Neural Net

In [41]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [42]:
# Neural Net model
# Define the basic neural network model
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=12, activation="relu", input_dim=21))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
29/29 [==============================] - 0s 570us/step - loss: 1.1608 - accuracy: 0.1721
Epoch 2/100
29/29 [==============================] - 0s 641us/step - loss: 0.8514 - accuracy: 0.4145
Epoch 3/100
29/29 [==============================] - 0s 606us/step - loss: 0.6494 - accuracy: 0.6447
Epoch 4/100
29/29 [==============================] - 0s 605us/step - loss: 0.5194 - accuracy: 0.7774
Epoch 5/100
29/29 [==============================] - 0s 570us/step - loss: 0.4354 - accuracy: 0.8454
Epoch 6/100
29/29 [==============================] - 0s 606us/step - loss: 0.3792 - accuracy: 0.8936
Epoch 7/100
29/29 [==============================] - 0s 606us/step - loss: 0.3391 - accuracy: 0.9167
Epoch 8/100
29/29 [==============================] - 0s 606us/step - loss: 0.3101 - accuracy: 0.9287
Epoch 9/100
29/29 [==============================] - 0s 606us/step - loss: 0.2878 - accuracy: 0.9408
Epoch 10/100
29/29 [==============================] - 0s 641us/step - loss: 0.2702 - accura

29/29 [==============================] - 0s 606us/step - loss: 0.0825 - accuracy: 0.9704
Epoch 82/100
29/29 [==============================] - 0s 606us/step - loss: 0.0822 - accuracy: 0.9704
Epoch 83/100
29/29 [==============================] - 0s 641us/step - loss: 0.0817 - accuracy: 0.9704
Epoch 84/100
29/29 [==============================] - 0s 570us/step - loss: 0.0810 - accuracy: 0.9715
Epoch 85/100
29/29 [==============================] - 0s 606us/step - loss: 0.0811 - accuracy: 0.9715
Epoch 86/100
29/29 [==============================] - 0s 570us/step - loss: 0.0806 - accuracy: 0.9715
Epoch 87/100
29/29 [==============================] - 0s 606us/step - loss: 0.0801 - accuracy: 0.9715
Epoch 88/100
29/29 [==============================] - 0s 606us/step - loss: 0.0799 - accuracy: 0.9715
Epoch 89/100
29/29 [==============================] - 0s 606us/step - loss: 0.0795 - accuracy: 0.9715
Epoch 90/100
29/29 [==============================] - 0s 606us/step - loss: 0.0797 - accuracy: 

## Deep Learning

In [44]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 12
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
29/29 [==============================] - 0s 641us/step - loss: 0.4815 - accuracy: 0.8739
Epoch 2/100
29/29 [==============================] - 0s 641us/step - loss: 0.3195 - accuracy: 0.9605
Epoch 3/100
29/29 [==============================] - 0s 606us/step - loss: 0.2402 - accuracy: 0.9638
Epoch 4/100
29/29 [==============================] - 0s 641us/step - loss: 0.1997 - accuracy: 0.9638
Epoch 5/100
29/29 [==============================] - 0s 641us/step - loss: 0.1766 - accuracy: 0.9638
Epoch 6/100
29/29 [==============================] - 0s 606us/step - loss: 0.1610 - accuracy: 0.9638
Epoch 7/100
29/29 [==============================] - 0s 641us/step - loss: 0.1487 - accuracy: 0.9638
Epoch 8/100
29/29 [==============================] - 0s 641us/step - loss: 0.1393 - accuracy: 0.9638
Epoch 9/100
29/29 [==============================] - 0s 605us/step - loss: 0.1321 - accuracy: 0.9638
Epoch 10/100
29/29 [==============================] - 0s 606us/step - loss: 0.1261 - accura

29/29 [==============================] - 0s 641us/step - loss: 0.0567 - accuracy: 0.9792
Epoch 82/100
29/29 [==============================] - 0s 641us/step - loss: 0.0558 - accuracy: 0.9792
Epoch 83/100
29/29 [==============================] - 0s 606us/step - loss: 0.0558 - accuracy: 0.9792
Epoch 84/100
29/29 [==============================] - 0s 606us/step - loss: 0.0556 - accuracy: 0.9792
Epoch 85/100
29/29 [==============================] - 0s 606us/step - loss: 0.0571 - accuracy: 0.9748
Epoch 86/100
29/29 [==============================] - 0s 606us/step - loss: 0.0551 - accuracy: 0.9781
Epoch 87/100
29/29 [==============================] - 0s 606us/step - loss: 0.0547 - accuracy: 0.9803
Epoch 88/100
29/29 [==============================] - 0s 606us/step - loss: 0.0536 - accuracy: 0.9792
Epoch 89/100
29/29 [==============================] - 0s 606us/step - loss: 0.0536 - accuracy: 0.9803
Epoch 90/100
29/29 [==============================] - 0s 606us/step - loss: 0.0533 - accuracy: 